<a href="https://colab.research.google.com/github/KubaSiwiec/plant_recognition/blob/main/resize_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import json
import numpy as np
import os
import pandas as pd
import torch
import random
 
from PIL import Image
from scipy import integrate
from scipy import misc
from scipy import stats
from skimage import img_as_float
from skimage import io
from sklearn import preprocessing

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def load_image_and_preprocess(path, segmented_path):
    # Open image from disk
    image = cv2.imread(path.strip())
    segmented_image = cv2.imread(segmented_path.strip())
 
    img = segmented_image
    h, w = img.shape[:2]
    height, width = h, w
    imggray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # print(type(img))
    # print('Height: {:3d}, Width: {:4d}\n'.format(height,width))
    ret, thresh = cv2.threshold(imggray, 127, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Calculate bounding rectangles for each contour.
    rects = [cv2.boundingRect(cnt) for cnt in contours]
    if rects == []:
        # No contours in image (pure black) just keep entire image
        top_y = 0
        bottom_y = height - 120
        left_x = 0
        right_x = width0 = -120
    else:
        # Calculate the combined bounding rectangle points.
        top_y = max(0, min([y for (x, y, w, h) in rects]) - 40)
        bottom_y = min(height, max([y + h for (x, y, w, h) in rects]) + 80)
        left_x = max(0, min([x for (x, y, w, h) in rects]) - 40)
        right_x = min(width - 1, max([x + w for (x, y, w, h) in rects]) + 80)
 
        # Prevent Tile Out of Bounds Issues
        if top_y == bottom_y:
            bottom_y = min(height, bottom_y + 1)
            top_y = max(0, top_y - 1)
        if left_x == right_x:
            right_x = min(width, right_x + 1)
            left_x = max(0, left_x - 1)
 
        # Landscape Image
        if width >= height:
            if left_x >= 450:
                right_x = width - 150
                left_x = 0
            if top_y >= 350 or bottom_y <= 200:
                top_y = 0
                bottom_y = height - 100
        # Portrait
        else:
            if left_x >= 350:
                right_x = width - 100
                left_x = 0
            if top_y >= 450 or bottom_y <= 250:
                top_y = 0
                bottom_y = height - 150
 
    # Use the rectangle to crop on original image
    img = image[top_y:bottom_y, left_x:right_x]
    img_pil = Image.fromarray(img)
    img_pil = img_pil.resize((256, 256))
    # print(img_pil.size)
    return img_pil

In [ ]:
from PIL import Image
import os
import os.path
import PIL
import torch
from torchvision import datasets, models, transforms
from glob import glob
from os.path import dirname, basename, join, exists
from pathlib import Path
 
# image = Image.open(image_path)
 
# #
# left = (width - new_width)/2
# top = (height - new_height)/2
# right = (width + new_width)/2
# bottom = (height + new_height)/2
 
# Crop the center of the image
# im = im.crop((left, top, right, bottom))
 
# resized_image = image.resize((300,300))
 
# save_path = '/content/gdrive/MyDrive/ny1157-01-1-resized.pt'
 
# print(resized_image.size)
# print(image.size)
 
trans = transforms.Compose([
        transforms.CenterCrop(600),
        transforms.Resize(300)
        # transforms.ToTensor(),
        # transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
 
 
 
# tensor_img = transforms.ToTensor(resized_image)
# tensor_img = trans(resized_image)
# print(tensor_img.shape)
# torch.save(tensor_img, save_path)
 
# resized_image.save(save_path)
 
image_dir = '/content/gdrive/MyDrive/leafsnap_data/leafsnap-dataset/dataset/images/*/*'
classes = []
location = {}
source = '' # lab or field 
 
save_path = '/content/gdrive/MyDrive/leafsnap_data/leafsnap-dataset-resized-no-sep'
lab_path = join(save_path, 'lab')
field_path = join(save_path, 'field')
Path(save_path).mkdir(parents=True, exist_ok=True)
Path(lab_path).mkdir(parents=True, exist_ok=True)  
Path(field_path).mkdir(parents=True, exist_ok=True)
i = 1 # to track progress
 
bad_lab_species = set(['abies_concolor', 'abies_nordmanniana', 'picea_pungens', 'picea_orientalis',
                       'picea_abies', 'cedrus_libani', 'cedrus_atlantica', 'cedrus_deodara',
                       'juniperus_virginiana', 'tsuga_canadensis', 'larix_decidua', 'pseudolarix_amabilis', 'ulmus_procera'])
 
for class_dir in glob(image_dir):
  if basename(class_dir) in bad_lab_species:
    print("Skipping {}".format(basename(class_dir)))
    continue
  Path(class_save_path).mkdir(parents=True, exist_ok=True)
  source = basename(dirname(class_dir))
  class_save_path = join(save_path, basename(class_dir)) #join(save_path, join(source, basename(class_dir)))
  print(class_save_path)
  
 
  classes.append(basename(class_dir))
  
  print(basename(class_dir))
 
  images_resized = 0
 
  for image in glob(class_dir + '/*'):
    # print(image)
    segmented_path = image.replace('/images/', '/segmented/').replace('jpg', 'png')
    # print(segmented_path)
    # pil_image = Image.open(image)
    # tensor_image = trans(pil_image)
    # print(exists(segmented_path))
    # print(exists(image))
    if exists(segmented_path):
      # print("Exists")
      resized_image = load_image_and_preprocess(image, segmented_path)
      image_save_path = join(class_save_path, basename(image))
      resized_image.save(image_save_path)
      images_resized += 1
 
    # tensor_image.save(resized_image)
    # torch.save(tensor_image, image_save_path)
  print("Images resized: {}".format(images_resized))
 
  print("Class " + str(i))
  i += 1
 
# print(classes)

Skipping abies_concolor
Skipping abies_nordmanniana
/content/gdrive/MyDrive/leafsnap_data/leafsnap-dataset-resized-no-sep/acer_campestre
acer_campestre
Images resized: 36
Class 1
/content/gdrive/MyDrive/leafsnap_data/leafsnap-dataset-resized-no-sep/acer_ginnala
acer_ginnala
Images resized: 31
Class 2
/content/gdrive/MyDrive/leafsnap_data/leafsnap-dataset-resized-no-sep/acer_griseum
acer_griseum
Images resized: 46
Class 3
/content/gdrive/MyDrive/leafsnap_data/leafsnap-dataset-resized-no-sep/acer_negundo
acer_negundo
Images resized: 33
Class 4
/content/gdrive/MyDrive/leafsnap_data/leafsnap-dataset-resized-no-sep/acer_palmatum
acer_palmatum
Images resized: 92
Class 5
/content/gdrive/MyDrive/leafsnap_data/leafsnap-dataset-resized-no-sep/acer_pensylvanicum
acer_pensylvanicum
Images resized: 35
Class 6
/content/gdrive/MyDrive/leafsnap_data/leafsnap-dataset-resized-no-sep/acer_platanoides
acer_platanoides
Images resized: 20
Class 7
/content/gdrive/MyDrive/leafsnap_data/leafsnap-dataset-resize